User Experience Analysis

In [1]:
import pickle
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
import os , sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from data_selector import *
from data_visualizer import *
from data_outlier_handler import OutlierHandler

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
pd.set_option('display.max_columns', 100)

Reading Data

In [8]:
df = pd.read_csv("../Data/cleaned_Tellco_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60586 entries, 0 to 60585
Data columns (total 55 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        60586 non-null  int64  
 1   Bearer Id                         60586 non-null  int64  
 2   Start                             60586 non-null  object 
 3   Start ms                          60586 non-null  float64
 4   End                               60586 non-null  object 
 5   End ms                            60586 non-null  float64
 6   IMSI                              60586 non-null  int64  
 7   MSISDN/Number                     60586 non-null  int64  
 8   IMEI                              60586 non-null  int64  
 9   Last Location Name                60586 non-null  object 
 10  Avg RTT DL (ms)                   60586 non-null  float64
 11  Avg RTT UL (ms)                   60586 non-null  float64
 12  Avg 

User Experience Metrics

In [9]:
user_experience_df = df[[
    'MSISDN/Number',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)',
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Handset Type']]

In [10]:
# caluclate the totals
user_experience_df['Total Avg RTT (ms)'] = user_experience_df['Avg RTT DL (ms)'] + user_experience_df['Avg RTT UL (ms)']
user_experience_df['Total Avg Bearer TP (kbps)'] = user_experience_df['Avg Bearer TP DL (kbps)'] + user_experience_df['Avg Bearer TP UL (kbps)']
user_experience_df['Total TCP Retrans. Vol (Bytes)'] = user_experience_df['TCP DL Retrans. Vol (Bytes)'] + user_experience_df['TCP UL Retrans. Vol (Bytes)']
user_experience_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60586 entries, 0 to 60585
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   MSISDN/Number                   60586 non-null  int64  
 1   Avg RTT DL (ms)                 60586 non-null  float64
 2   Avg RTT UL (ms)                 60586 non-null  float64
 3   Avg Bearer TP DL (kbps)         60586 non-null  float64
 4   Avg Bearer TP UL (kbps)         60586 non-null  float64
 5   TCP DL Retrans. Vol (Bytes)     60586 non-null  float64
 6   TCP UL Retrans. Vol (Bytes)     60586 non-null  float64
 7   Handset Type                    60586 non-null  object 
 8   Total Avg RTT (ms)              60586 non-null  float64
 9   Total Avg Bearer TP (kbps)      60586 non-null  float64
 10  Total TCP Retrans. Vol (Bytes)  60586 non-null  float64
dtypes: float64(9), int64(1), object(1)
memory usage: 5.1+ MB


In [11]:
user_experience_df.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33763490140,39.0,18.0,3698.0,416.0,19520.0,7230.0,unknown,57.0,4114.0,26750.0
1,33659219748,97.0,7.0,3845.0,2535.0,3231397.0,7230.0,Samsung Galaxy Note 8 (Sm-N950F Ds),104.0,6380.0,3238627.0
2,33664473872,217.0,4.0,28305.0,6500.0,5722628.0,312007.0,Huawei B593S-22,221.0,34805.0,6034635.0
3,33659219748,79.0,4.0,4148.0,4540.0,32754526.0,34638.0,Samsung Galaxy Note 8 (Sm-N950F Ds),83.0,8688.0,32789164.0
4,33753758738,39.0,33.0,263.0,620.0,12964929.0,882193.0,Samsung Galaxy J3 (Sm-J330),72.0,883.0,13847122.0


In [12]:
# select the necessary columns
user_experience_df = user_experience_df[['MSISDN/Number', 'Handset Type', 'Total Avg RTT (ms)',
    'Total Avg Bearer TP (kbps)', 'Total TCP Retrans. Vol (Bytes)']]
user_experience_df.head()

,MSISDN/Number,Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33763490140,unknown,57.0,4114.0,26750.0
1,33659219748,Samsung Galaxy Note 8 (Sm-N950F Ds),104.0,6380.0,3238627.0
2,33664473872,Huawei B593S-22,221.0,34805.0,6034635.0
3,33659219748,Samsung Galaxy Note 8 (Sm-N950F Ds),83.0,8688.0,32789164.0
4,33753758738,Samsung Galaxy J3 (Sm-J330),72.0,883.0,13847122.0


In [13]:
# aggregating user experience metrics per user
user_experience_df1 = user_experience_df.groupby(
    'MSISDN/Number').agg({
        'Total Avg RTT (ms)': 'sum',
        'Total Avg Bearer TP (kbps)': 'sum',
        'Total TCP Retrans. Vol (Bytes)': 'sum',
        'Handset Type': [lambda x: x.mode()[0]]})
user_experience_df1.head()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
,sum,sum,sum,<lambda>
MSISDN/Number,,,,
33601007832,84.0,248.0,2396.0,Apple iPhone 5S (A1457)
33601008617,91.0,56781.0,9370832.0,Apple iPhone Se (A1723)
33601010682,24.0,7787.0,3190.0,Samsung Galaxy A8 (2018)
33601011634,39.0,42416.0,110232.0,Huawei Mate 10 Pro Porsche Design Huawei Mate 10
33601011959,52.0,1247.0,11569.0,Samsung Galaxy S8 Plus (Sm-G955F)


In [14]:
user_experience_df = pd.DataFrame(columns=[
    "Total Avg RTT (ms)",
    "Total Avg Bearer TP (kbps)",
    "Total TCP Retrans. Vol (Bytes)",
    "Handset Type"])

user_experience_df["Total Avg RTT (ms)"] = user_experience_df1["Total Avg RTT (ms)"]['sum']
user_experience_df["Total Avg Bearer TP (kbps)"] = user_experience_df1["Total Avg Bearer TP (kbps)"]['sum']
user_experience_df["Total TCP Retrans. Vol (Bytes)"] = user_experience_df1["Total TCP Retrans. Vol (Bytes)"]['sum']
user_experience_df["Handset Type"] = user_experience_df1["Handset Type"]['<lambda>']
user_experience_df.head()

,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
MSISDN/Number,,,,
33601007832,84.0,248.0,2396.0,Apple iPhone 5S (A1457)
33601008617,91.0,56781.0,9370832.0,Apple iPhone Se (A1723)
33601010682,24.0,7787.0,3190.0,Samsung Galaxy A8 (2018)
33601011634,39.0,42416.0,110232.0,Huawei Mate 10 Pro Porsche Design Huawei Mate 10
33601011959,52.0,1247.0,11569.0,Samsung Galaxy S8 Plus (Sm-G955F)


Top 10 values of the experience metrics

In [15]:
# top 10 rtt values in the dataset

rtt = user_experience_df.sort_values('Total Avg RTT (ms)', ascending=False)
top_10 = rtt.head(10)['Total Avg RTT (ms)']
bottom_10 = rtt.tail(10)['Total Avg RTT (ms)']
most_10 = user_experience_df['Total Avg RTT (ms)'].value_counts().head(10)

In [17]:
multi_hist([top_10, bottom_10, most_10], 1, 3,
    "RTT values in the dataset", ['Top 10', 'Bottom 10', 'Most 10'])

In [18]:
# top 10 tp values in the dataset

tp = user_experience_df.sort_values('Total Avg Bearer TP (kbps)', ascending=False)
top_10 = tp.head(10)['Total Avg Bearer TP (kbps)']
bottom_10 = tp.tail(10)['Total Avg Bearer TP (kbps)']
most_10 = user_experience_df['Total Avg Bearer TP (kbps)'].value_counts().head(10)

In [19]:
multi_hist([top_10, bottom_10, most_10], 1, 3,
    "TP values in the dataset", ['Top 10', 'Bottom 10', 'Most 10'])

In [20]:
# top 10 tcp values in the dataset

tcp = user_experience_df.sort_values('Total TCP Retrans. Vol (Bytes)', ascending=False)
top_10 = tcp.head(10)['Total TCP Retrans. Vol (Bytes)']
bottom_10 = tcp.tail(10)['Total TCP Retrans. Vol (Bytes)']
most_10 = user_experience_df['Total TCP Retrans. Vol (Bytes)'].value_counts().head(10)

In [22]:
multi_hist([top_10, bottom_10, most_10], 1, 3,
    "TCP values in the dataset", ['Top 10', 'Bottom 10', 'Most 10'])